<a href="https://colab.research.google.com/github/DanBaissa/symbolic_regression_polisci/blob/main/Symbolic_Regression_Political_Science_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbolic Regression in Political Science: Classic Models

---

## Overview

This educational notebook demonstrates how **symbolic regression**—a machine learning technique that automatically discovers mathematical equations—can help political scientists and social scientists uncover the underlying functional relationships in data. Instead of pre-selecting a model form (like linear or quadratic), symbolic regression explores a huge space of possible equations and finds the one(s) that best fit the observed data. This makes it especially useful for:

- Detecting non-linearities, thresholds, and cycles that traditional regressions may miss.
- Revealing interaction effects and complex dynamics.
- Providing **interpretable formulas** instead of black-box predictions.

The notebook combines classic social science and political economy data-generating processes (linear, quadratic, interaction, cyclical) and demonstrates how symbolic regression can discover these relationships using two tools:

- Python’s `gplearn` (genetic programming)
- Julia’s `PySR` (highly efficient, often simpler equations)

For each dataset, you’ll see:

- The context and political science analogy
- The discovered formula (compared to the “ground truth”)
- The test-set fit (R²)
- Example paper-style writeups for interpreting your findings

## 1. Installation and Imports

In [ ]:
!pip install gplearn pysr julia
import julia
julia.install()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

## 2. Data Generation: Classic Relationships

In [ ]:
rng = np.random.default_rng(42)
X = np.linspace(-3, 3, 200).reshape(-1, 1)

# Classic models
y_linear = 2.5 * X.ravel() + 3 + rng.normal(0, 0.1, X.shape[0])
y_quadratic = -1.2 * (X.ravel() ** 2) + 2 * X.ravel() + 1 + rng.normal(0, 0.1, X.shape[0])
X2 = rng.uniform(-3, 3, X.shape[0])
y_interaction = 1.5 * X.ravel() * X2 + 0.5 * X2 + rng.normal(0, 0.1, X.shape[0])
y_sinusoidal = 1.8 * np.sin(1.5 * X.ravel()) + 0.2 * X.ravel() + rng.normal(0, 0.1, X.shape[0])

datasets = [
    ("Linear", X, y_linear, None),
    ("Quadratic", X, y_quadratic, None),
    ("Interaction", np.column_stack([X.ravel(), X2]), y_interaction, X2),
    ("Sinusoidal", X, y_sinusoidal, None)
]

## 3. Symbolic Regression with Python (`gplearn`)

In [ ]:
gplearn_results = []

for name, X_data, y_data, _ in datasets:
    print(f"\n=== {name} (Python/gplearn) ===")
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)
    py_est = SymbolicRegressor(
        population_size=1000, generations=20, stopping_criteria=0.01,
        p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
        p_point_mutation=0.1, max_samples=0.9, verbose=0,
        parsimony_coefficient=0.01, random_state=0
    )
    py_est.fit(X_train, y_train)
    y_pred_py = py_est.predict(X_test)
    py_r2 = r2_score(y_test, y_pred_py)
    py_eq = str(py_est._program)
    print(f"Discovered equation (Python): {py_eq}")
    print(f"Test set R^2: {py_r2:.3f}")
    gplearn_results.append({'name': name, 'eq': py_eq, 'r2': py_r2})

## 4. Symbolic Regression: Democracy and Redistribution (Applied Example)

### A. Merge the Data

In [ ]:
df_wb = pd.read_csv("world_bank.csv")
df_polity = pd.read_csv("polity.csv")
df = pd.merge(df_wb, df_polity, left_on=['country_code','year'], right_on=['scode','year'], how='inner')
df = df.dropna(subset=['inf_mort', 'polity', 'gdp_per_capita', 'year', 'pop_density'])

### B. Symbolic Regression: Does Democracy Predict Infant Mortality?

In [ ]:
X = df[['polity']].values
y = df['inf_mort'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

sr = SymbolicRegressor(population_size=500, generations=10, stopping_criteria=0.01,
                       p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
                       p_point_mutation=0.1, max_samples=0.9, verbose=1,
                       parsimony_coefficient=0.01, random_state=0)
sr.fit(X_train, y_train)

print("Discovered equation:", sr._program)
y_pred = sr.predict(X_test)
print("Test set R^2:", r2_score(y_test, y_pred))

### C. Symbolic Regression: All Controls

In [ ]:
X_ctrl = df[['polity', 'gdp_per_capita', 'year', 'pop_density']].values
y_ctrl = df['inf_mort'].values

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_ctrl, y_ctrl, test_size=0.3, random_state=1)

sr_ctrl = SymbolicRegressor(population_size=500, generations=10, stopping_criteria=0.01,
                            p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
                            p_point_mutation=0.1, max_samples=0.9, verbose=1,
                            parsimony_coefficient=0.01, random_state=0)
sr_ctrl.fit(Xc_train, yc_train)

print("Discovered equation (with controls):", sr_ctrl._program)
yc_pred = sr_ctrl.predict(Xc_test)
print("Test set R^2:", r2_score(yc_test, yc_pred))

### D. Symbolic Regression: Country Fixed Effects (Optional)

In [ ]:
N = 8
country_counts = df['country_name'].value_counts()
top_countries = country_counts.index[:N]
df_fe = df[df['country_name'].isin(top_countries)]

enc = OneHotEncoder(drop='first', sparse=False)
country_dummies = enc.fit_transform(df_fe[['country_name']])

X_fe = np.concatenate([df_fe[['polity', 'gdp_per_capita', 'year', 'pop_density']].values, country_dummies], axis=1)
y_fe = df_fe['inf_mort'].values

Xfe_train, Xfe_test, yfe_train, yfe_test = train_test_split(X_fe, y_fe, test_size=0.3, random_state=1)

sr_fe = SymbolicRegressor(population_size=300, generations=6, stopping_criteria=0.05,
                          p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
                          p_point_mutation=0.1, max_samples=0.9, verbose=1,
                          parsimony_coefficient=0.01, random_state=0)
sr_fe.fit(Xfe_train, yfe_train)

print("Discovered equation (w/ country FE dummies):", sr_fe._program)
yfe_pred = sr_fe.predict(Xfe_test)
print("Test set R^2:", r2_score(yfe_test, yfe_pred))